In [ ]:
from diffusers import DiffusionPipeline, AutoencoderKL
import torch

In [ ]:
device = "cuda"  # fp16 cannot be run on CPU

Models are being loaded from:
https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0

In [ ]:
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix", 
    torch_dtype=torch.float16
)
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)
pipe.to(device)

In [ ]:
prompt = "A man in a spacesuit is running a marathon in the jungle."

image = pipe(prompt=prompt, num_inference_steps=25, num_images_per_prompt=4)